In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

In [ ]:
path = 'drive/My Drive/股票'
db = sqlite3.connect(path + '/twstock_indie.db')
df = pd.read_sql(con=db, sql='select * from "8163"')
df.head()

,index,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Date
0,1008,達方,"846,000",583,"36,237,350",43.15,43.15,42.60,42.80,-,0.55,42.80,6,42.85,6,7.60,2019-06-03 00:00:00
1,1008,達方,"833,212",597,"35,737,766",43.00,43.30,42.65,42.75,-,0.05,42.75,70,42.85,13,7.59,2019-06-04 00:00:00
2,1008,達方,"1,326,714",943,"56,647,984",43.25,43.30,42.05,42.20,-,0.55,42.20,8,42.25,3,7.50,2019-06-05 00:00:00
3,1008,達方,"1,273,380",795,"53,344,150",42.20,42.40,41.70,41.75,-,0.45,41.75,3,41.80,1,7.42,2019-06-06 00:00:00
4,1008,達方,"987,975",751,"41,860,629",42.10,42.70,41.90,42.60,+,0.85,42.60,1,42.65,24,7.57,2019-06-10 00:00:00


In [ ]:
import math

df.index = pd.to_datetime(df['Date'])
df = df[['開盤價','最高價','最低價','收盤價']]
df.columns = ['Open','High','Low','Close']
cols = df.columns
df[cols] = df[cols].apply(pd.to_numeric)


In [ ]:
df.head()

,Open,High,Low,Close
Date,,,,
2019-06-03,43.15,43.15,42.60,42.80
2019-06-04,43.00,43.30,42.65,42.75
2019-06-05,43.25,43.30,42.05,42.20
2019-06-06,42.20,42.40,41.70,41.75
2019-06-10,42.10,42.70,41.90,42.60


> ## backtest

In [ ]:
!pip install Backtesting
!pip install backtrader

In [ ]:
from backtesting import Backtest
from backtesting.lib import SignalStrategy
import pandas as pd


class Strategy(SignalStrategy):
    
    n1 = 5
    n2 = 10
    
    def init(self):
        super().init()
        
        # Precompute the two moving averages
        close =  pd.Series(self.data.Close)
        sma1 = close.rolling(self.n1).mean()
        sma2 = close.rolling(self.n2).mean()

        # Precompute signal
        signal_long = (sma1 > sma2) & (sma1.shift() < sma2.shift())
        signal_short = (sma1 < sma2) & (sma1.shift() > sma2.shift())
        
        # combine signal
        signal = signal_long
        signal[signal_short] = -1
        
        # add signal
        self.set_signal(signal)


bt = Backtest(df, Strategy)
result1 = bt.run()
bt.plot()